In [1]:
import keras
import tensorflow_hub as hub
import pathlib
import tensorflow as tf
import os, shutil

from keras import layers

Using TensorFlow backend.


## Download the Pretrained Classifier

Via Keras

In [0]:
# base_model = tf.keras.applications.MobileNetV2(input_shape=(150, 150, 3),
#                                               include_top=False,
#                                               weights='imagenet')

Via Tensorflow Hub

In [3]:
# Image information
HEIGHT = 224
WIDTH = 224
CHANNELS = 3

feature_extractor_url = "https://tfhub.dev/google/imagenet/inception_v3/classification/3" #@param {type:"string"}

module = hub.Module(feature_extractor_url, tags=['train'])
feature_extractor_layer = hub.KerasLayer(module,
                                         input_shape=(HEIGHT, WIDTH, CHANNELS))

feature_extractor_layer.trainable = False


model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

W0820 10:47:23.196473 140102132995968 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1001)              23853833  
_________________________________________________________________
flatten (Flatten)            (None, 1001)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               256512    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 24,110,602
Trainable params: 256,769
Non-trainable params: 23,853,833
_________________________________________________________________


In [0]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

## Download the Data

In [5]:
URL = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"

zip_file = tf.keras.utils.get_file(origin=URL,
                                   fname="cats_and_dogs_filtered.zip", 
                                   extract=True)
base_dir, _ = os.path.splitext(zip_file)

68608000/68606236 [==============================] - 0s 0us/step


In [6]:
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')
print ('Total training cat images:', len(os.listdir(train_cats_dir)))

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')
print ('Total training dog images:', len(os.listdir(train_dogs_dir)))

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
print ('Total validation cat images:', len(os.listdir(validation_cats_dir)))

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
print ('Total validation dog images:', len(os.listdir(validation_dogs_dir)))

Total training cat images: 1000
Total training dog images: 1000
Total validation cat images: 500
Total validation dog images: 500


## Preprocess the Data

In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(HEIGHT, WIDTH),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(HEIGHT, WIDTH),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


## Fit the Model

In [9]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

Epoch 1/10


KeyboardInterrupt: ignored

## Visualisation

In [0]:
import matplotlib.pyplot as plt

# Retrieve a list of accuracy results on training and test datasets for each training epoch
acc = history.history['acc']
val_acc = history.history['val_acc']

# Retrieve a list of list results on training and test datasets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']


# Get number of epochs
epochs = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc, label='Training acc')
plt.plot(epochs, val_acc, label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()